In [2]:
class Game():
    
    def __init__(self, square_length):
        self.square_length = 3
        self.side_length = self.square_length*self.square_length
        self.current_sub_square = 0
        self.board= [0][0]
        self.sub_squares = {}
        self.rows = {}
        self.columns = {}
        self.temp_sub_squares = {}
        self.temp_columns = {}
        self.num_found = False
        self.restart = False
        self.iterations = 0
        self.entries = 0

    def gameSetup(self):

        self.gameSetupRestart()

        while(self.entries<self.side_length*self.side_length):
            for row_num in range(self.side_length):
                col_num = 0
                while(col_num<self.side_length):
                    self.current_sub_square = math.floor(row_num/self.square_length)*self.square_length + math.floor(col_num/self.square_length)
                    num_found = False

                    while(not num_found):
                        temp = []

                        for num in self.rows[row_num]:
                            if num not in temp and num in self.columns[col_num] and num in self.sub_squares[self.current_sub_square]:
                                temp.append(num)

                        if(len(temp)==0):
                            self.sub_squares = self.subSquareReset(self.sub_squares, self.current_sub_square, row_num)
                            self.rows = self.rowReset(self.rows, row_num)
                            self.columns = self.columnReset(self.columns, col_num)
                            self.boardRowReset(row_num)
                            col_num = 0
                            self.current_sub_square = math.floor(row_num/self.square_length)*self.square_length + math.floor(col_num/self.square_length)
                            self.iterations+=1

                            if(self.iterations>self.side_length*self.side_length):
                                self.restart = True
                                break
                        else:
                            seed = random.randint(0, len(temp)-1)
                            value = temp[seed]
                            self.board[row_num][col_num] = value
                            self.temp_columns[col_num].append(value)
                            self.temp_sub_squares[self.current_sub_square][math.floor(row_num%self.square_length)].append(value)
                            self.sub_squares[self.current_sub_square].remove(value)
                            self.rows[row_num].remove(value)
                            self.columns[col_num].remove(value)
                            self.entries+=1
                            col_num+=1
                            num_found = True

                    if(self.restart):
                        break

                for m in range(self.side_length):
                    self.temp_columns[m] = []

                if(self.restart):
                    self.gameSetupRestart()
                    break

        if(not self.boardChecker()):
            self.gameSetup()
        else:
            for i in range(self.side_length):
                for j in range(self.side_length):
                    self.board[i][j]+=1                                      

    def gameSetupRestart(self):
        
        self.entries = 0
        self.iterations = 0
        self.restart = False
        self.board = [[-1 for i in range(self.side_length)] for i in range(self.side_length)]

        for i in range(self.side_length):
            self.sub_squares[i] = []
            self.rows[i] = []
            self.columns[i] = []
            self.temp_sub_squares[i] = {}
            self.temp_columns[i] = []

        for j in range(self.side_length):
            for k in range(self.side_length):
                self.sub_squares[j].append(k)
                self.rows[j].append(k)
                self.columns[j].append(k)

            for m in range(self.side_length):
                for n in range(self.side_length):
                    self.temp_sub_squares[m][n] = []

    def subSquareReset(self, m, sub_square, row_num):
        
        num_to_clear = sub_square%self.square_length
        sub_row = row_num%self.square_length

        for i in range(num_to_clear+1):
            temp = copy.deepcopy(self.temp_sub_squares[sub_square-i][sub_row])
            m[sub_square-i].extend(temp)
            self.temp_sub_squares[sub_square-i][sub_row] = []

        return m

    def rowReset(self, m, row_num):
        
        m[row_num] = []

        for i in range(self.side_length):
            m[row_num].append(i)
        return m

    def columnReset(self, m, col_num):
        
        for i in range(col_num):
            for num in self.temp_columns[i]:
                m[i].append(num)

            self.temp_columns[i] = []

        return m

    def boardRowReset(self, row_num):
        for k in range(self.side_length):
            self.board[row_num][k] = -1

    def boardChecker(self):
        for i in range(self.side_length):
            for j in range(self.side_length):
                for k in range(j+1, self.side_length):
                    if self.board[i][j]==self.board[i][k] or self.board[j][i]==self.board[k][i]:
                        return False
        self.entries = 0
        row = 0
        col = 0
        modifier = 0
        values = {}

        for p in range(self.side_length):
            values[p] = []

        while(self.entries<self.side_length*self.side_length):
            for i in range(row, row+self.square_length):
                for j in range(col, col+self.square_length):
                    self.entries+=1

                    if len(values[self.board[i][j]])==0:
                        values[self.board[i][j]].append(1)
                    else:
                        return False

            for m in range(self.side_length):
                values[m] = []

            modifier+=1

            if modifier%self.square_length==0:
                col = 0
                row+=self.square_length
            else:
                col+=self.square_length

        return True
    
    def returnBoard(self):
        return self.board

    def printBoard(self):
        
        for i in range(self.side_length):
            for j in range(self.side_length):
                print(self.board[i][j], end=" ")
            print(' ')
        print(' ')

In [3]:
class AI():

    def __init__(self, game_board, square_length):
        self.game_board = game_board
        self.square_length = square_length
        self.side_length = square_length*square_length
        self.legal_moves = {}
        self.legal_copy = {}
        
        for i in range(self.side_length):
            self.legal_moves[i] = {}
            
            for j in range(self.side_length):
                if self.game_board[i][j]==0:
                    self.legal_moves[i][j] = []
                    for k in range(1, self.side_length+1):
                        self.legal_moves[i][j].append(k)
                        
    def removeValues(self, board, moves):
        
        move_made = False
        
        for row in range(self.side_length):
            for col in range(self.side_length):
                if(board[row][col]!=0):
                    current_sub_square = math.floor(row/self.square_length)*self.square_length + math.floor(col/self.square_length)
                    
                    for num in range(self.side_length):
                        if(board[row][num]==0 and board[row][col] in moves[row][num]):
                            moves[row][num].remove(board[row][col])
                            
                        if(board[num][col]==0 and board[row][col] in moves[num][col]):
                            moves[num][col].remove(board[row][col])
                            
                        for s in range(self.side_length):
                            if(board[num][s]==0 and (math.floor(num/self.square_length)*self.square_length + math.floor(s/self.square_length))==current_sub_square and board[row][col] in moves[num][s]):
                                moves[num][s].remove(board[row][col])
        
        return move_made
    
    def onlyLeft(self, board, moves, verifying):
        success = False
        
        for i in range(self.side_length):
            for j in range(self.side_length):
                if(board[i][j]==0 and len(moves[i][j])==1):
                    board[i][j] = moves[i][j][0]
                    moves[i][j] = []
                    
                    if(verifying):
                        return True
                    else:
                        success = True
        
        return success
    
    def checkRow(self, board, moves, verifying):
        
        num_left = {}
        success = False
        
        for row in range(self.side_length):
            for i in range(1, self.side_length+1):
                num_left[i] = 0
            
            for col in range(self.side_length):
                for num in range(1, self.side_length+1):
                    if(board[row][col]==0 and num in moves[row][col]):
                        val = num_left[num]
                        num_left[num] = val+1
                
            for j in range(1, self.side_length+1):
                if num_left[j]==1:
                    for c in range(self.side_length):
                        if(board[row][c]==0 and j in moves[row][c]):
                            board[row][c] = j
                            moves[row][c] = []
                            
                            if(verifying):
                                return True
                            else:
                                success = True
        
        return success
    
    def checkColumn(self, board, moves, verifying):
        
        num_left = {}
        success = False
        
        for col in range(self.side_length):
            for i in range(1, self.side_length+1):
                num_left[i] = 0
            
            for row in range(self.side_length):
                for num in range(1, self.side_length+1):
                    if(board[row][col]==0 and num in moves[row][col]):
                        val = num_left[num]
                        num_left[num] = val+1
                
            for j in range(1, self.side_length):
                if num_left[j]==1:
                    for r in range(self.side_length):
                        if(board[r][col]==0 and j in moves[r][col]):
                            board[r][col] = j
                            moves[r][col] = []
                            
                            if(verifying):
                                return True
                            else:
                                success = True
        
        return success
    
    def checkSubSquare(self, board, moves, verifying):
        
        num_left = {}
        success = False

        for i in range(0, self.side_length):
            num_left[i] = {}
            
            for j in range(1, self.side_length+1):
                num_left[i][j] = 0
        
        for col in range(self.side_length):
            for row in range(self.side_length):
                current_sub_square = math.floor(row/self.square_length)*self.square_length + math.floor(col/self.square_length)
                
                for num in range(1, self.side_length+1):
                    if(board[row][col]==0 and num in moves[row][col]):
                        val = num_left[current_sub_square][num]
                        num_left[current_sub_square][num] = val+1
        
        for ss in range(self.side_length):
            for j in range(1, self.side_length):
                if num_left[ss][j]==1:
                    for r in range(self.side_length):
                        for c in range(self.side_length):
                            current_sub_square = math.floor(r/self.square_length)*self.square_length + math.floor(c/self.square_length)
                        
                            if(board[r][c]==0 and current_sub_square==ss and j in moves[r][c]):
                                board[r][c] = j
                                moves[r][c] = []
                                
                                if(verifying):
                                    return True
                                else:
                                    success = True
        
        
        return success
    
    def makeMoves(self, board, moves, verifying):
        
        success = False
        self.removeValues(board, moves)
        
        if self.checkRow(board, moves, verifying):
            self.removeValues(board, moves)
            success = True
            
        if self.checkColumn(board, moves, verifying):
            self.removeValues(board, moves)
            success = True
            
        if self.checkSubSquare(board, moves, verifying):
            self.removeValues(board, moves)
            success = True
            
        if(not self.onlyLeft(board, moves, verifying) and not success):
            return False
        
        return True
    
    def tryMove(self, board, verifying):
        
        success = False
        completed_board = False
        
        for row in range(self.side_length):
            for col in range(self.side_length):
                if board[row][col]==0:
                    remove_nums = []
                    
                    for num in self.legal_moves[row][col]:
                        legal_copy = copy.deepcopy(self.legal_moves)
                        temp_board = copy.deepcopy(self.game_board)
                        temp_board[row][col] = num
                        
                        while(self.makeMoves(board, legal_copy, verifying)):
                            if(self.attemptFailed(temp_board, legal_copy) and verifying):
                                remove_nums.append(num)
                                success = True
                            
                            if(verifying and self.isGameOver(temp_board)):
                                if(not completed_board):
                                    completed_board = True
                                else:
                                    return False
                            
                            if(not verifying and self.isGameOver(temp_board)):
                                self.game_board = copy.deepcopy(temp_board)
                                
                    for num in remove_nums:
                        self.legal_moves[row][col].remove(num)
                        
        return success
    
    def attemptFailed(self, board, moves):
        for i in range(self.side_length):
            for j in range(self.side_length):
                if(board[i][j]==0 and len(moves[i][j])==0):
                    return True
                
        return False
    
    def iterateBoard(self, board, verifying):
        
        while(not self.isGameOver(board) and self.makeMoves(board, self.legal_moves, verifying)):
            pass
            
        if(not self.isGameOver(self.game_board)):
            return False

        return True
    
    def playGame(self, board, verifying):
        while(not self.isGameOver(board)):
            if(not self.iterateBoard(board, verifying)):
                if(not self.tryMove(board, verifying)):
                    return False
        
        if(self.isGameOver(board)):
            return True
    
        return False
    
    def isGameOver(self, board):
        for row in range(self.side_length):
            for col in range(self.side_length):
                if board[row][col]==0:
                    return False
                
        return True
    
    def returnBoard(self):
        return self.game_board
    
    def printBoard(self):
        for i in range(self.side_length):
            for j in range(self.side_length):
                print(self.game_board[i][j], end=" ")
            print(' ')
        print(' ')

In [4]:
class GameBoard():

    def __init__(self, filled_board, square_length, difficulty):
        
        self.filled_board = filled_board
        self.square_length = square_length
        self.side_length = square_length*square_length
        self.difficulty = difficulty
        self.game_board = [0][0]
                     
    def makeGame(self):
        
        threshold = 0;
        self.game_board = [[0 for i in range(self.side_length)] for i in range(self.side_length)]
        
        if(self.difficulty=='h'):
            threshold = (self.square_length-1)/self.square_length - (1/(self.square_length+1))
        else:
            threshold = (self.square_length-1)/self.square_length - (1/(self.square_length+2))
            
        for row in range(self.side_length):
            for col in range(self.side_length):
                x = random.uniform(0, 1)
                if(x<threshold):
                    self.game_board[row][col] = self.filled_board[row][col]
                    
        self.ambiguityFinder()
  
        test_game_board = [[0 for i in range(self.side_length)] for i in range(self.side_length)]
        for i in range(self.side_length):
            test_game_board[i] = copy.deepcopy(self.game_board[i])
            
        ai = AI(test_game_board, self.square_length)
        
        if(not ai.playGame(test_game_board, True)):
            self.makeGame()
            
        return self.game_board
    
    def ambiguityFinder(self):
        current_sub_square = 0
        arr = [-1]*(self.side_length-1)
        
        for m in range(1, self.side_length):
            arr[m-1] = m
            
        for row in range(self.side_length-1):
            for col in range(self.side_length-1):
                current_sub_square = math.floor(row/self.square_length)*self.square_length + math.floor(col/self.square_length)
                
                if self.game_board[row][col]==0:
                    for num in arr:
                        if((row+num)<self.side_length and self.game_board[row+num][col]==0 and current_sub_square==((row+num)/self.square_length)*self.square_length + col/self.square_length):
                            for j in range(col+1, self.side_length):
                                if(self.game_board[row][j]==0 and self.game_board[row+num][j]==0):
                                    
                                    x = random.randint(0, 3)
                                    
                                    if(x==0):
                                        self.game_board[row][col] = self.filled_board[row][col]
                                    elif(x==1):
                                        self.game_board[row+num][col] = self.filled_board[row+num][col]
                                    elif(x==2):
                                        self.game_board[row][j] = self.filled_board[row][j]
                                    else:
                                        self.game_board[row+num][j] = self.filled_board[row+num][j]
                                        
                        if((col+num)<self.side_length and self.game_board[row][col+num]==0 and current_sub_square==((row)/self.square_length)*self.square_length + (col+num)/self.square_length):
                            for j in range(row+1, self.side_length):
                                if(self.game_board[j][col]==0 and self.game_board[j][col+num]==0):
                                    
                                    x = random.randint(0, 3)
                                    
                                    if(x==0):
                                        self.game_board[row][col] = self.filled_board[row][col]
                                    elif(x==1):
                                        self.game_board[row][col+num] = self.filled_board[row][col+num]
                                    elif(x==2):
                                        self.game_board[j][col] = self.filled_board[j][col]
                                    else:
                                        self.game_board[j][col+num] = self.filled_board[j][col+num]                               
                                        
    def returnBoard(self):
        return self.game_board
                
    def printBoard(self):
        for i in range(self.side_length):
            for j in range(self.side_length):
                print(self.game_board[i][j], end=" ")
            print(' ')
        print(' ')

In [5]:
import pygame as pg
from pygame.locals import *
import sys
import time
import math
import copy
import random
import datetime
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#initialize global variables
cell_num = 0
checked = None
draw = False
width = 400
height = 400
white = (255, 255, 255)
line_color = (10,10,10)
#TicTacToe 3x3 board
square_length = 3
side_length = square_length*square_length
g = Game(square_length)
g.gameSetup()
g.printBoard()
filled_board = g.returnBoard()

gb = GameBoard(filled_board, square_length, 'h')
game_board = gb.makeGame()
print('Game Board')
gb.printBoard()
board = gb.returnBoard()
original_board = copy.deepcopy(board)


pg.init()
fps = 30
CLOCK = pg.time.Clock()
screen = pg.display.set_mode((width, height), pg.RESIZABLE)
pg.display.set_caption('Sudoku')
errors = 0

    
def isGameOver():
    global board, filled_board
    
    for i in range(side_length):
        for j in range(side_length):
            if(board[i][j]!=filled_board[i][j]):
                return False
    return True
    
def userClick():
    #get coordinates of mouse click
    global checked, errors
    
    x, y = pg.mouse.get_pos()
    width = screen.get_width()
    height = screen.get_height()
    dim = int(min(screen.get_width(), screen.get_height()))
    board_dim = dim

    if width > height:
        if width > height*1.5:
            board_dim = dim
        else:
            board_dim = 2*width/3
    elif width <= height:
        if width*1.5 < height:
            board_dim = dim
        else:
            board_dim = 2*height/3

    col = math.ceil(x/(board_dim/9))
    row = math.ceil(y/(board_dim/9))
    
    if(row>0 and col>0 and row<10 and col<10):
        if checked==(row-1, col-1):
            posx1 = int(board_dim*(col-1)/9)+board_dim/200
            posy1 = int(board_dim*(row-1)/9)+board_dim/200
            posx2 = int(board_dim*col/9)-board_dim/200
            posy2 = int(board_dim*row/9)-board_dim/200

            font = pg.font.Font(None, int(board_dim/10))

            screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

            if board[row-1][col-1]!=0:
                for i in range(side_length):
                    for j in range(side_length):
                        if(board[i][j]==board[row-1][col-1]):
                            posx1 = int(board_dim*(j)/9)+board_dim/200
                            posy1 = int(board_dim*(i)/9)+board_dim/200
                            posx2 = int(board_dim*(j+1)/9)-board_dim/200
                            posy2 = int(board_dim*(i+1)/9)-board_dim/200

                            screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                            posx = board_dim/9*j+board_dim/25
                            posy = board_dim/9*i+board_dim/30
                            font = pg.font.Font(None, int(board_dim/10))

                            if original_board[row-1][col-1]!=0:  # Black if in 
                                text = font.render(str(board[row-1][col-1]), 1, (0, 0, 0))
                            else:
                                text = font.render(str(board[row-1][col-1]), 1, (128, 128, 128)) 

                            screen.blit(text, (posx, posy)) 
                
                screen.blit(text, (posx, posy))

            checked = None    
            return [True]
        else:
            if checked!=None:
                posx1 = int(board_dim*(checked[1])/9)+board_dim/200
                posy1 = int(board_dim*(checked[0])/9)+board_dim/200
                posx2 = int(board_dim*(checked[1]+1)/9)-board_dim/200
                posy2 = int(board_dim*(checked[0]+1)/9)-board_dim/200

                font = pg.font.Font(None, int(board_dim/10))

                screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
            
                    
                if(board[checked[0]][checked[1]]!=0):
                    for i in range(side_length):
                        for j in range(side_length):
                            if(board[i][j]==board[checked[0]][checked[1]]):
                                posx1 = int(board_dim*(j)/9)+board_dim/200
                                posy1 = int(board_dim*(i)/9)+board_dim/200
                                posx2 = int(board_dim*(j+1)/9)-board_dim/200
                                posy2 = int(board_dim*(i+1)/9)-board_dim/200

                                screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                                posx = board_dim/9*j+board_dim/25
                                posy = board_dim/9*i+board_dim/30
                                font = pg.font.Font(None, int(board_dim/10))

                                if original_board[i][j]!=0:  # Black if in 
                                    text = font.render(str(board[checked[0]][checked[1]]), 1, (0, 0, 0))
                                else:
                                    text = font.render(str(board[checked[0]][checked[1]]), 1, (128, 128, 128)) 

                                screen.blit(text, (posx, posy))

            checked = (row-1, col-1)
            
            posx1 = int(board_dim*(col-1)/9)+board_dim/200
            posy1 = int(board_dim*(row-1)/9)+board_dim/200
            posx2 = int(board_dim*col/9)-board_dim/200
            posy2 = int(board_dim*row/9)-board_dim/200
            
            if(board[row-1][col-1]==0):  # Filling in blank square with pink
                screen.fill((255, 204, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
                
                
                
            else:   # Creating yellow background for all squares with selected number
                for i in range(side_length):
                    for j in range(side_length):
                        if(board[i][j]==board[row-1][col-1]):
                            posx1 = int(board_dim*(j)/9)+board_dim/200
                            posy1 = int(board_dim*(i)/9)+board_dim/200
                            posx2 = int(board_dim*(j+1)/9)-board_dim/200
                            posy2 = int(board_dim*(i+1)/9)-board_dim/200

                            screen.fill((255, 255, 204), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                            posx = board_dim/9*j+board_dim/25
                            posy = board_dim/9*i+board_dim/30
                            font = pg.font.Font(None, int(board_dim/10))

                            if original_board[row-1][col-1]!=0:  # Black if in 
                                text = font.render(str(board[row-1][col-1]), 1, (0, 0, 0))
                            else:
                                text = font.render(str(board[row-1][col-1]), 1, (128, 128, 128)) 

                            screen.blit(text, (posx, posy))

    found_num = False
    
    if checked!=None:
        if height >= width:
            if(y>(board_dim+(height-board_dim)/2) and x<=board_dim):
                num_selected = math.ceil(x/(board_dim/9))
                found_num = True
        else:
            if(x>(board_dim+(width-board_dim)/2) and y<=board_dim):
                num_selected = math.ceil(y/(board_dim/9))
                found_num = True
                
        if found_num:
            posx1 = int(board_dim*(checked[1])/9)+board_dim/200
            posy1 = int(board_dim*(checked[0])/9)+board_dim/200
            posx2 = int(board_dim*(checked[1]+1)/9)-board_dim/200
            posy2 = int(board_dim*(checked[0]+1)/9)-board_dim/200

            font = pg.font.Font(None, int(board_dim/10))
            screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

            posx = board_dim/9*checked[1]+board_dim/25
            posy = board_dim/9*checked[0]+board_dim/30

            if(filled_board[checked[0]][checked[1]]==num_selected):
                font = pg.font.Font(None, int(board_dim/10))
                text = font.render(str(num_selected), 1, (128, 128, 128))
                screen.blit(text, (posx, posy))

                board[checked[0]][checked[1]] = num_selected
                checked = None
            else:
                screen.fill((255, 0, 0), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
                checked = None
                errors+=1
                return [False, posx1, posy1, posx2, posy2]
            
    return [True]

def userPress():
    #get coordinates of mouse click
    global checked, errors
    
    num_selected = None
    
    if checked!=None:
        if pg.key.get_pressed()[K_1]==1 or pg.key.get_pressed()[K_KP1]==1:
            num_selected = 1
        elif pg.key.get_pressed()[K_2]==1 or pg.key.get_pressed()[K_KP2]==1:
            num_selected = 1
        elif pg.key.get_pressed()[K_3]==1 or pg.key.get_pressed()[K_KP3]==1:
            num_selected = 3
        elif pg.key.get_pressed()[K_4]==1 or pg.key.get_pressed()[K_KP4]==1:
            num_selected = 4
        elif pg.key.get_pressed()[K_5]==1 or pg.key.get_pressed()[K_KP5]==1:
            num_selected = 5
        elif pg.key.get_pressed()[K_6]==1 or pg.key.get_pressed()[K_KP6]==1:
            num_selected = 6
        elif pg.key.get_pressed()[K_7]==1 or pg.key.get_pressed()[K_KP7]==1:
            num_selected = 7
        elif pg.key.get_pressed()[K_8]==1 or pg.key.get_pressed()[K_KP8]==1:
            num_selected = 8
        elif pg.key.get_pressed()[K_9]==1 or pg.key.get_pressed()[K_KP9]==1:
            num_selected = 9

        if num_selected!=None:
            width = screen.get_width()
            height = screen.get_height()
            dim = int(min(screen.get_width(), screen.get_height()))
            board_dim = dim

            if width > height:
                if width > height*1.5:
                    board_dim = dim
                else:
                    board_dim = 2*width/3
            elif width <= height:
                if width*1.5 < height:
                    board_dim = dim
                else:
                    board_dim = 2*height/3

            row = checked[0]
            col = checked[1]

            posx1 = int(board_dim*(checked[1])/9)+board_dim/200
            posy1 = int(board_dim*(checked[0])/9)+board_dim/200
            posx2 = int(board_dim*(checked[1]+1)/9)-board_dim/200
            posy2 = int(board_dim*(checked[0]+1)/9)-board_dim/200

            font = pg.font.Font(None, int(board_dim/10))
            screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

            posx = board_dim/9*checked[1]+board_dim/25
            posy = board_dim/9*checked[0]+board_dim/30

            if(filled_board[checked[0]][checked[1]]==num_selected):
                font = pg.font.Font(None, int(board_dim/10))
                text = font.render(str(num_selected), 1, (128, 128, 128))
                screen.blit(text, (posx, posy))

                board[checked[0]][checked[1]] = num_selected
                checked = None
            else:
                screen.fill((255, 0, 0), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
                checked = None
                errors+=1
                return [False, posx1, posy1, posx2, posy2]

    return [True]
        
def CreateWindow(width=400, height=600):
    '''Updates the window width and height '''
    global checked
    
    screen = pg.display.set_mode((width, height), RESIZABLE)
    pg.display.update()
    screen.fill(white)
    dim = int(min(width, height))
    board_dim = dim
    
    if width > height:
        if width > height*1.5:
            board_dim = dim
        else:
            board_dim = 2*width/3
    elif width <= height:
        if width*1.5 < height:
            board_dim = dim
        else:
            board_dim = 2*height/3
        
        board_dim+(height-board_dim)/2
    
    # Drawing vertical lines
    for num in range(10):
        line_thick = 1

        if num%3==0:
            line_thick = 3
        
        # Vertical lines of board
        pg.draw.line(screen, line_color, (board_dim*num/9.0, 0), (board_dim*num/9.0, board_dim), line_thick)
        # Horizontal Lines of board
        pg.draw.line(screen, line_color, (0, board_dim*num/9.0), (board_dim, board_dim*num/9.0), line_thick)
        
        if height>=width:  # Game buttons on bottom if height greater than or equal width
            pg.draw.line(screen, line_color, (board_dim*num/9.0, board_dim+(height-board_dim)/2), (board_dim*num/9.0, height), 1)
        else:  # Game buttons on side if width greater height
            pg.draw.line(screen, line_color, (board_dim+(width-board_dim)/2, board_dim*num/9.0), (width, board_dim*num/9.0), 1)

        if(checked!=None):
            if(board[checked[0]][checked[1]]!=0):
                for i in range(side_length):
                    for j in range(side_length):
                        if(board[i][j]==board[checked[0]][checked[1]]):
                            posx1 = int(board_dim*(j)/9)+board_dim/200
                            posy1 = int(board_dim*(i)/9)+board_dim/200
                            posx2 = int(board_dim*(j+1)/9)-board_dim/200
                            posy2 = int(board_dim*(i+1)/9)-board_dim/200

                            screen.fill((255, 255, 204), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                            posx = board_dim/9*j+board_dim/25
                            posy = board_dim/9*i+board_dim/30
                            font = pg.font.Font(None, int(board_dim/10))

                            if original_board[i][j]!=0:  # Black if in 
                                text = font.render(str(board[checked[0]][checked[1]]), 1, (0, 0, 0))
                            else:
                                text = font.render(str(board[checked[0]][checked[1]]), 1, (128, 128, 128)) 

                            screen.blit(text, (posx, posy))
            else:
                posx1 = int(board_dim*checked[1]/9)+board_dim/200
                posy1 = int(board_dim*checked[0]/9)+board_dim/200
                posx2 = int(board_dim*(checked[1]+1)/9)-board_dim/200
                posy2 = int(board_dim*(checked[0]+1)/9)-board_dim/200
                
                font = pg.font.Font(None, int(board_dim/10))
                screen.fill((255, 204, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1]) 

        for n in range(10):
            if board[num-1][n-1]!=0:
                posx = board_dim/9*(n-1)+board_dim/25
                posy = board_dim/9*(num-1)+board_dim/30
                
                font = pg.font.Font(None, int(board_dim/10))
                text = font.render(str(board[num-1][n-1]), 1, (0, 0, 0))
                screen.blit(text, (posx, posy))
    
    if height>=width:
        pg.draw.line(screen, line_color, (0, board_dim+(height-board_dim)/2), (board_dim, board_dim+(height-board_dim)/2), 1)
        pg.draw.line(screen, line_color, (0, height), (board_dim, height), 1)
        
        for num in range(1, 10):
            posx = board_dim/9*(num-1)+board_dim/25
            posy = board_dim+3*(height-board_dim)/4
            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(num), 1, (0, 0, 0))
            screen.blit(text, (posx, posy))
    else:
        pg.draw.line(screen, line_color, (board_dim+(width-board_dim)/2, 0), (board_dim+(width-board_dim)/2, board_dim), 1)
        pg.draw.line(screen, line_color, (width, 0), (width, board_dim), 1)
        
        for num in range(1, 10):
            posx = board_dim+3*(width-board_dim)/4
            posy = board_dim/9*(num-1)+board_dim/25
            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(num), 1, (0, 0, 0))
            screen.blit(text, (posx, posy))
        
def resetGame():
    global board, winner, cell_num, draw
    
    g = Game(square_length)
    g.gameSetup()
    g.printBoard()
    filled_board = g.returnBoard()

    gb = GameBoard(filled_board, square_length, 'h')
    game_board = gb.makeGame()
    print('Game Board')
    gb.printBoard()
    board = gb.returnBoard()
    

CreateWindow()
CLOCK = pg.time.Clock()
wrong_num_data = [None]
start_time = time.time()
t = datetime.datetime.now().strftime('%H:%M:%S')

# run the game loop forever
while(True):
    seconds = str(int((time.time()-start_time)%60))
    if(len(seconds)==1):
        seconds = '0' + seconds
    pg.display.set_caption(str(math.floor((time.time()-start_time)/60)) + ':' + seconds + '       errors: ' + str(errors))
    if(wrong_num_data[0]!=None and (pg.time.get_ticks()-wrong_num_data[0]>500)):  # Filling wrong board space with white
        screen.fill((255, 255, 255), rect=[wrong_num_data[1], wrong_num_data[2], wrong_num_data[3], wrong_num_data[4]])
        wrong_num_data = [None]
        
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == VIDEORESIZE:
            CreateWindow(event.w, event.h)
        elif event.type is MOUSEBUTTONDOWN:
            # the user clicked; place an X or O
            u = userClick()
            if(not u[0]): # If user picked wrong number, set data in list to time of wrong number and coordinates
                wrong_num_data = [copy.deepcopy(pg.time.get_ticks()), u[1], u[2], u[3]-u[1], u[4]-u[2]]
        elif event.type is KEYDOWN:
            
            # the user clicked; place an X or O
            u = userPress()
            if(not u[0]): # If user picked wrong number, set data in list to time of wrong number and coordinates
                wrong_num_data = [copy.deepcopy(pg.time.get_ticks()), u[1], u[2], u[3]-u[1], u[4]-u[2]]
                
        if(isGameOver()):
            resetGame()
            CreateWindow()
            
    pg.display.update()
    CLOCK.tick(fps)

3 9 7 2 8 1 5 6 4  
5 6 4 7 9 3 1 8 2  
2 8 1 6 5 4 9 7 3  
1 5 6 9 3 7 4 2 8  
7 4 8 5 6 2 3 1 9  
9 2 3 4 1 8 7 5 6  
4 3 2 8 7 5 6 9 1  
6 1 5 3 2 9 8 4 7  
8 7 9 1 4 6 2 3 5  
 
Game Board
0 0 7 2 8 0 5 0 0  
5 0 4 7 0 3 0 0 2  
0 8 0 0 5 4 9 7 0  
1 5 0 0 3 7 4 0 8  
0 4 8 0 0 2 3 1 0  
0 0 3 4 0 0 7 0 0  
4 0 0 0 7 0 6 9 1  
6 1 5 3 0 0 0 0 0  
8 7 9 0 0 6 2 0 0  
 


SystemExit: 

C:\Users\Jack\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
import datetime
t = datetime.datetime.now().strftime('%H:%M:%S')

print(datetime.datetime.now().strftime('%H:%M:%S')-t)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [136]:
square_length = 3
side_length = square_length*square_length
wins = 0

for game in range(1):
    won_game = True

    g = Game(square_length)
    g.gameSetup()
    g.printBoard()
    filled_board = g.returnBoard()

    gb = GameBoard(filled_board, square_length, 'h')
    game_board = gb.makeGame()
    print('Game Board')
    gb.printBoard()

    #ai = AI(game_board, square_length)
    #ai.playGame(game_board, False)
    #print("Game Over")
    #ai.printBoard()

    for row in range(side_length):
        for col in range(side_length):
            if(ai.returnBoard()[row][col]!=filled_board[row][col] and ai.returnBoard[row][col]!=0):
                won_game = False

    if(won_game):
        wins+=1
    
print(wins)

6 4 8 7 2 5 3 1 9  
3 5 2 1 4 9 6 8 7  
9 7 1 8 6 3 2 5 4  
7 2 6 5 1 4 9 3 8  
1 9 3 2 7 8 4 6 5  
4 8 5 9 3 6 7 2 1  
5 3 7 6 9 1 8 4 2  
2 1 4 3 8 7 5 9 6  
8 6 9 4 5 2 1 7 3  
 
Game Board
0 4 0 0 2 5 0 1 9  
3 5 2 1 0 9 6 0 0  
9 7 1 8 6 3 2 5 4  
0 2 6 5 0 4 0 3 0  
1 9 3 0 7 8 4 6 5  
4 8 5 0 3 6 7 0 0  
5 0 7 6 0 1 0 4 0  
2 1 0 3 8 0 5 9 6  
8 6 9 4 5 2 0 7 3  
 


TypeError: 'method' object is not subscriptable